# 🔹 What is Apache Spark?

Apache Spark is an open-source distributed computing system used for big data processing and analytics. It’s known for its speed, ease of use, and ability to process large-scale data.

### Key Concepts:
**Distributed computing:** Spark processes data in parallel across many machines.

**In-memory processing:** Unlike Hadoop, Spark can keep data in memory for faster access.

**Unified engine:** Supports multiple tasks like batch processing, streaming, machine learning, and SQL.

### Spark Ecosystem Components:

| **Component**     | **Description** |
|-------------------|-----------------|
| **Spark SQL**     | Run SQL queries on large datasets. Supports structured and semi-structured data. |
| **Spark Core**    | The foundation – handles basic I/O, scheduling, and task execution. |
| **Spark Streaming** | For processing real-time data streams. |


# 🔹 Parallelism
### 🍬 Real-Life Analogy: Counting M&M Lentils with Spark

Imagine you're organizing a big event and you receive a **huge bag of mixed M&M lentils**—millions of them.

Your goal? 👉 Count how many M&Ms there are of each color.

You don't want to do this alone—it would take forever. So, you call your friends (Spark Cluster) to help you out!

---

#### 🧠 The Spark Cluster Analogy

| **Spark Component** | **Real-Life Equivalent** |
|---------------------|--------------------------|
| **Driver Program**  | You (the event organizer giving instructions) |
| **Executors**       | Your friends helping count M&Ms |
| **Cluster Manager** | Your group chat deciding who does what |
| **Tasks**           | "Count the red ones from this bucket" |
| **Partitions**      | Separate bowls of M&Ms to divide the work |

---

#### 🛠️ How the Work is Distributed

1. **Driver Program (You)**:  
   You write down a plan: "Count how many red, blue, green, yellow M&Ms there are."

2. **Divide the Work (Partition the Data)**:  
   You pour the big bag into 4 bowls (data partitions), one for each friend.

3. **Send Tasks to Executors**:  
   You give each friend a simple task:  
   "Count how many of each color you find in your bowl."

4. **Friends Start Counting (Parallel Processing)**:  
   All friends count at the same time – no waiting!  
   Each friend returns a small result like:  
   - Friend 1: Red – 105, Blue – 87, Green – 94  
   - Friend 2: Red – 98, Blue – 102, Green – 101  
   *(...and so on)*

5. **Combine the Results (Aggregation at Driver)**:  
   You collect all the partial counts and add them together:  
   - Total Red = 105 + 98 + ...  
   - Total Blue = 87 + 102 + ...

---

#### ✅ Spark Benefits Shown Here:

- **Speed**: Counting is done in parallel.
- **Scalability**: You can invite more friends (add executors) if the bag gets bigger.
- **Efficiency**: You only do the final summation (aggregation).

---

#### 📌 In Real Spark SQL

You might run:
```sql
SELECT color, COUNT(*) FROM m_and_ms GROUP BY color


# 🔹 Partitioning
---
So how can we help with efficiency and scalability within code ? first answer is propper table partitioning.  
### 🧩 Table Partitioning: Distributing Work Evenly

Continuing our M&M analogy 🍬...

Let’s say you again have 4 friends helping to count M&Ms. But this time, you accidentally pour 90% of the M&Ms into **one** bowl, and only small amounts into the other 3.

Now what happens?

- One friend is **overwhelmed** with M&Ms. 😩
- The other three finish quickly and **sit idle**. 🪑
- The final result takes much longer than necessary. 🕒

This is called **data skew** — when some partitions have much more data than others.

---

### 🔍 What is Table Partitioning?

In Spark (and Delta Lake / Parquet), **partitioning** means physically organizing data files **by column values**, typically in the file system (e.g., `/table/date=2025-07-18/`).

Partitioning allows Spark to:

- **Prune unnecessary data** (read only what's needed)
- **Parallelize processing efficiently**
- **Avoid scanning full tables**

---

### 🧠 Good Partitioning Example

If you're querying a table by **`event_date`** most of the time, you could partition by `event_date`.

That way, when running:
 

```%sql
SELECT * FROM events WHERE event_date = '2025-07-18'```

.. you can skip chunks of data which you are not interested in and ease memory utilization ( instead of reading it all and filtering , you can just pick what you need).

### ⚠️ What Happens with Bad Partitioning?

Let’s explore **two common partitioning mistakes**:

---

#### ❌ 1. Partitioning by a Skewed Column

Let’s say you partition by `user_id`, and one user (e.g., `user_42`) has 90% of the activity.

- Spark writes most of the data into one partition (`user_id=42`).
- That partition takes much longer to process.
- Other partitions are idle, causing **data skew** and **slow jobs**.

---

#### ❌ 2. Partitioning by a Unique Key (e.g., Primary Key)

Let’s say you partition by `transaction_id` or `order_id` — which are **unique for each row**.

- Spark creates **one folder per row** (partition).
- You end up with **millions of small files** and folders.
- Metadata operations become slow (e.g., listing files, updating table state).
- Queries are slower because Spark must scan lots of tiny files.
- Writes can fail or severely degrade due to **filesystem overhead**.

📉 This is called the **small file problem** — and it kills performance at scale.

---

### 🧠 Key Insight

> **Just because a column is important (like a primary key) doesn’t mean it should be used for partitioning.**

---

### ✅ Better Alternatives

| Instead of this...                | Try this...                        |
|----------------------------------|------------------------------------|
| Partitioning by `transaction_id` | Don’t partition, or use `event_date` |
| Partitioning by `user_id`        | Use `country`, `region`, or `user_type` if more balanced |
| No partitioning at all           | Partition by columns in common filters (e.g., `event_date`, `category`) |

---

### 📦 Summary

> Partitioning helps **divide the work**, but poor partitioning creates more problems than it solves.

Always aim for:
- Balanced partition sizes
- Not too many partitions
- Predictable, filterable values

Spark loves **"just right" partitioning** — not too many, not too few.


# 🔹 Excercise 1.
- create dataframe/temporary view with your username so its unique 
- create 2 tables
  - partitioned by primary key
  - correctly partitioned by event_date
- check table definitions
- observe load times when reading from tables  
- clear resources

In [0]:
%python
from pyspark.sql.functions import expr
from datetime import datetime

# build name for your table
table_name = spark.sql("select replace(split(current_user(), '@')[0],'.','') AS short_name").collect()[0]['short_name']
table_base = f"gmsgq_training_{table_name}_sales"

# Create demo DataFrame
data = spark.range(0, 10000).selectExpr(
    "id as order_id",
    "CAST(id % 50 AS STRING) AS user_id",  # 50 users
    "date_sub(current_date(), CAST(id % 10 AS INT)) AS event_date",
    "CAST(rand() * 100 AS DECIMAL(10,2)) AS amount"
)

data.createOrReplaceTempView("tmp_sales")

print(f"Base table name: {table_base}")


In [0]:
CREATE TABLE ${table_base}_good_partitioning
USING DELTA
PARTITIONED BY (event_date) -- partitioned by event_date (well distributed across workers, alows to be pruned when filtered by where statement)
AS
SELECT * FROM tmp_sales;

In [0]:
DESCRIBE table ${table_base}_good_partitioning;

In [0]:
CREATE TABLE ${table_base}_bad_partitioning 
USING DELTA
PARTITIONED BY (order_id) -- partitioned by primary Key ( big problem)
AS 
SELECT
  *
FROM
  tmp_sales;

In [0]:
DESCRIBE TABLE ${table_base}_bad_partitioning;

In [0]:
-- Filter by event_date (partition pruning works!)
SELECT COUNT(*) FROM ${table_base}_good_partitioning WHERE event_date = current_date();

In [0]:
-- Try filtering on order_id (no partition pruning)
SELECT COUNT(*) FROM ${table_base}_bad_partitioning WHERE order_id < 123;

In [0]:
drop table if exists ${table_base}_good_partitioning;
drop table if exists ${table_base}_bad_partitioning;